# Week 2 Assignment

In [10]:
import graphlab as gl
# Set product key on this computer. After running this cell, you will not need to re-enter your product key. 
graphlab.product_key.set_product_key('030E-977B-ECE8-621B-C981-277C-4FD1-72E7')

# Limit number of worker processes. This preserves system memory, which prevents hosted notebooks from crashing.
graphlab.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 4)

# Output active product key.
graphlab.product_key.get_product_key()

'030E-977B-ECE8-621B-C981-277C-4FD1-72E7'

### Load Data

In [5]:
sales = graphlab.SFrame('home_data.gl/')
sales.head(2)

Columns:
	id	str
	date	datetime
	price	int
	bedrooms	str
	bathrooms	str
	sqft_living	int
	sqft_lot	int
	floors	str
	waterfront	int
	view	int
	condition	int
	grade	int
	sqft_above	int
	sqft_basement	int
	yr_built	int
	yr_renovated	int
	zipcode	str
	lat	float
	long	float
	sqft_living15	float
	sqft_lot15	float

Rows: 2

Data:
+------------+---------------------------+--------+----------+-----------+-------------+
|     id     |            date           | price  | bedrooms | bathrooms | sqft_living |
+------------+---------------------------+--------+----------+-----------+-------------+
| 7129300520 | 2014-10-13 00:00:00+00:00 | 221900 |    3     |     1     |     1180    |
| 6414100192 | 2014-12-09 00:00:00+00:00 | 538000 |    3     |    2.25   |     2570    |
+------------+---------------------------+--------+----------+-----------+-------------+
+----------+--------+------------+------+-----------+-------+------------+---------------+
| sqft_lot | floors | waterfront | view | condition | grade | sqft_above | sqft_basement |
+----------+--------+------------+------+-----------+-------+------------+---------------+
|   5650   |   1    |     0      |  0   |     3     |   7   |    1180    |       0       |
|   7242   |   2    |     0      |  0   |     3     |   7   |    2170    |      400      |
+----------+--------+------------+------+-----------+-------+------------+---------------+
+----------+--------------+---------+-------------+---------------+---------------+-----+
| yr_built | yr_renovated | zipcode |     lat     |      long     | sqft_living15 | ... |
+----------+--------------+---------+-------------+---------------+---------------+-----+
|   1955   |      0       |  98178  | 47.51123398 | -122.25677536 |     1340.0    | ... |
|   1951   |     1991     |  98125  | 47.72102274 |  -122.3188624 |     1690.0    | ... |
+----------+--------------+---------+-------------+---------------+---------------+-----+
[2 rows x 21 columns]

### 1. Selection and summary statistics: We found the zip code with the highest average house price. What is the average house price of that zip code?

In [22]:
avg_price_zipcode = sales.groupby('zipcode', gl.aggregate.MEAN('price'))
avg_price_zipcode.head(5)
avg_price_zipcode.sort('Avg of price', ascending=False)

zipcode,Avg of price
98039,2160606.6
98004,1355927.09779
98040,1194230.00355
98112,1095499.36803
98102,901258.238095
98109,879623.623853
98105,862825.231441
98006,859684.763052
98119,849448.01087
98005,810164.880952


### 2. Filtering data: What fraction of the houses have living space between 2000 sq.ft. and 4000 sq.ft.?

In [50]:
sqft_living_filtered = sales[(sales['sqft_living'] > 2000) & (sales['sqft_living'] < 4000)]
len(sqft_living_filtered)

9111

In [51]:
len(sales)

21613

### 3. Building a regression model with several more features: What is the difference in RMSE between the model trained with my_features and the one trained with advanced_features?

In [58]:
train_data,test_data = sales.random_split(.8,seed=0)

In [56]:
my_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'zipcode']

In [59]:
my_features_model = graphlab.linear_regression.create(train_data,target='price',features=my_features,validation_set=None)

In [61]:
print my_features_model.evaluate(test_data)

{'max_error': 3486584.509381705, 'rmse': 179542.4333126903}


In [68]:
advanced_features = my_features + ['waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']

In [69]:
advanced_features

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'zipcode',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated']

In [70]:
advanced_features_model = graphlab.linear_regression.create(train_data,target='price',features=advanced_features,validation_set=None)

Linear regression:
--------------------------------------------------------
Number of examples          : 17384
Number of features          : 14
Number of unpacked features : 14
Number of coefficients    : 123
Starting Newton Method
--------------------------------------------------------
+-----------+----------+--------------+--------------------+---------------+
| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
+-----------+----------+--------------+--------------------+---------------+
| 1         | 2        | 0.051464     | 3439893.596116     | 154900.530329 |
+-----------+----------+--------------+--------------------+---------------+
SUCCESS: Optimal solution found.



In [71]:
print my_features_model.evaluate(test_data)
print advanced_features_model.evaluate(test_data)

{'max_error': 3486584.509381705, 'rmse': 179542.4333126903}
{'max_error': 3511146.4853660734, 'rmse': 157107.78827078617}


In [72]:
179542.43-157107.78

22434.649999999994